In [26]:
import numpy as np

In [27]:
class Sigmoid:
    def __init__(self):
        self.params = []

    def forward(self,x):
        return 1 / (1 + np.exp(-x))

In [28]:
class Affine:
    def __init__(self,W,b):
        self.params = [W,b]
    def forward(self,x):
        W,b = self.params
        out = np.dot(x,W) + b
        return out        

In [29]:
class TwoLayerNet:
    def __init__(self, input_size,hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size


        W1 = np.random.randn(I,H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H,O)
        b2 = np.random.randn(O)

        self.layers = [Affine(W1,b1),Sigmoid(),Affine(W2,b2)]

        self.params = []

        for layer in self.layers:
            self.params += layer.params
    
    def predict(self,x):
        for layer in self.layers:
            x = layer.forward(x)
        return x        

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmax的输出
        self.t = None  # 监督标签

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 在监督标签为one-hot向量的情况下，转换为正确解标签的索引
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx
        

In [30]:
if __name__ == '__main__':
    x = np.random.randn(10,2)
    model = TwoLayerNet(2,4,3)
    s = model.predict(x)
    print(s)
    

[[-0.43233549 -0.18615193  0.99053962]
 [-0.19548861 -0.57392097  0.63942174]
 [ 0.52692237 -1.60062421 -0.47600719]
 [-0.19372425 -1.02988556  0.22311118]
 [-0.7920157   0.35031837  1.67638641]
 [-0.06862412 -0.90219744  0.33671566]
 [ 0.07386951 -1.44454041 -0.12969549]
 [-0.06644118 -0.62083292  0.55846856]
 [ 0.11202842 -0.62610021  0.49719985]
 [-0.58219685 -0.07263958  1.2628158 ]]
